In [1]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_03_unflipped'

# 01 - Begin Analysis

Enter the path to the dictionary generated by Notebook 01

In [2]:
dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_01_unflipped/tmp/dataset_dict_pruned_lsn_tms.json'

What correlation method would you like to use?
- Pearson is very fast, but depends on linearity and is sensitive to outliers
- Spearman is slower, but much more robust

In [3]:
correlation = 'spearman'

In [4]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
from calvin_utils.ccm_utils.stat_utils import CorrelationCalculator
data_loader = DataLoader(dict_path)
correlation_calculator = CorrelationCalculator(method=correlation, verbose=False)
corr_map_dict = correlation_calculator.generate_correlation_maps(data_loader)

Generate the Convergent Maps and Save the Individual Correlation Maps

In [ ]:
from calvin_utils.ccm_utils.ccm_utils import ConvergentMapGenerator
convergent_map_generator = ConvergentMapGenerator(corr_map_dict, data_loader, 
                                                  weight=False,
                                                  mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii',
                                                  out_dir=None)
convergent_map_generator.generate_and_save_maps()
convergent_map_generator.save_individual_r_maps()

# 02 - Use LOOCV to Train a Convergent Map on N-1 Datasets, Then Test it on the Nth dataset

Test the Convergent Map

In [ ]:
from calvin_utils.ccm_utils.ccm_utils import LOOCVAnalyzer
loocv_analyzer = LOOCVAnalyzer(corr_map_dict, data_loader, 
                               convergence_type='average', 
                               method='pearson', n_bootstrap=1000, 
                               out_dir=out_dir)
display(loocv_analyzer.results_df)

# 03 - Compute Fixed Effects Within Specific Groups

Print out datasets

Assign datasets to groups using a dictionary. Groups (values) allocate datasets to a fixed effect analysis.
```
{
'dataset_1': 'group1',
'dataset_2': 'group2',
'dataset_3': 'group1'
}
```

In [12]:
group_dict ={
    'adni_Alzheimer' : 'Fixed Effect Lesions',
    'corbetta_memory' : 'Fixed Effect Lesions',
    'manitoba_memory' : 'Fixed Effect Lesions',
    'ms_memory' : 'Fixed Effect Lesions',
    'hebscher_tms' : 'Fixed Effect Stim',
    'hermiller_tms_a1' : 'Fixed Effect Stim',
    'hermiller_tms_b' : 'Fixed Effect Stim',
    'wang_tms' : 'Fixed Effect Stim',
    }

Run It

In [ ]:
import pandas as pd
df = loocv_analyzer.compute_fixed_effects_by_group(group_dict)
meta_df = pd.concat([loocv_analyzer.results_df, df], ignore_index=True)

# 04 - Visualize Your Meta Analysis as a Forest Plot Figure 
- This is a meta-analysis. It presents the effect size (R/rho values) of your convergent map on the left-out map. 

Generate a Forest Plot of the Results

In [ ]:
from calvin_utils.ccm_utils.stat_utils import MetaConvergenceForestPlot
meta_forest_plot = MetaConvergenceForestPlot(meta_df, sig_digits=2, out_dir=out_dir, table=True)
meta_forest_plot.run()

That's all

-Calvin